# Generalized hourglass plots for MAF

## Setup of the notebook

### Imports

In [1]:
import warnings
import logging
import urllib
from os import path
from collections import OrderedDict
from tempfile import TemporaryDirectory
import matplotlib.pyplot as plt
from scipy.constants import golden

from rubin_sim import maf
import rubin_sim.maf.db
import rubin_sim.maf.metricBundles

In [2]:
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'
# %load_ext lab_black
# %load_ext pycodestyle_magic
# %flake8_on --ignore E501,W505
%load_ext autoreload
%autoreload 1

### `jupyter` magic

### Logging

Include logging to get a better idea of how long things take. This is more accurate than jupyter `%%time` magic, because it `%%time` does not include the time it takes for the client to render the plot, but this time is part of what is experienced by the user.

In [3]:
logging.basicConfig(format="%(asctime)s %(message)s")
logger = logging.getLogger("hourglass_notebook")
logger.setLevel("DEBUG")
logger.info("Starting")

2021-06-18 16:31:57,300 Starting


### Plotting configuration

The limited default resolution of matplotlib images can distort the gaps between exposures. This can be fixed either by increasing the precision of PNG image, or using a vector based format like SVG.
Comment out whichever jupyter magic below corresponds to the method you don't want to use.
For plots with lots of data, SVGs can cause performance issues in some browsers.

If you want to use SVGs, the line in this cell should be uncommented out:

In [4]:
# %config InlineBackend.figure_formats = ['svg']

If you want to use PNGs with high enough dpi to avoid pixel-level distortion, the lines in this cell can be uncommented out:

In [5]:
# plt.rcParams['figure.dpi'] = 300
# plt.rcParams['savefig.dpi'] = 300

In [6]:
plt.rcParams["backend"]

'module://matplotlib_inline.backend_inline'

Note that increasing the pdi in a jupyter notebook will make the images very large when displayed within the notebook.

### Suppress useless warnings

In [7]:
warnings.filterwarnings(
    "ignore",
    append=True,
    message=r".*Tried to get polar motions for times after IERS data is valid.*",
)
warnings.filterwarnings("ignore", append=True, message=r".*dubious year.*")

### Configuration

In [8]:
data_dir = "."

If you have a local copy of the scheduler output you want to use, set `opsim_origin` to its path.

In [9]:
opsim_origin = "https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs1.7/baseline/baseline_nexp2_v1.7_10yrs.db"
maf_output_dir = path.join(data_dir, "maf_output")

## Download the data file if necessary

In [10]:
if path.isfile(opsim_origin) or "opsim_fname" in locals():
    # Check for opsim_fname in locals so that if we rerun the notebook without restarting the kernel
    # it will not re-download the data
    opsim_fname = opsim_origin
else:
    opsim_data_dir = TemporaryDirectory(dir=data_dir)
    opsim_fname = path.join(opsim_data_dir.name, opsim_origin.split("/")[-1])
    logger.info("Downloading %s to %s", opsim_origin, opsim_fname)
    opsim_db_file = urllib.request.urlretrieve(opsim_origin, opsim_fname)
    logger.info("Download complete")

    print(
        f"Opsim data file downloaded to\n\t{opsim_fname}\nwhich will be deleted when the notebook is restarted or closed."
    )

2021-06-18 16:31:57,375 Downloading https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs1.7/baseline/baseline_nexp2_v1.7_10yrs.db to ./tmp4ghyg7c4/baseline_nexp2_v1.7_10yrs.db
2021-06-18 16:31:59,700 Download complete


Opsim data file downloaded to
	./tmp4ghyg7c4/baseline_nexp2_v1.7_10yrs.db
which will be deleted when the notebook is restarted or closed.


## Get input and output database connections

In [11]:
logger.debug("Configuring database connections")
ops_db = maf.db.OpsimDatabase(opsim_fname)
results_db = maf.db.ResultsDb(outDir=maf_output_dir)

2021-06-18 16:31:59,710 Configuring database connections


## Samples

### Median Seeing in 5 minute time intervals

Use the `TimeIntervaleSlicer` to divide the survey into 300 second (5 minute) time segments, and calculate the median seeing in each segment.
Make an hourglass plot of the result for October of 2023.

In [ ]:
breakpoint()
logger.info("Starting hourglass")
bundle_group = maf.metricBundles.MetricBundleGroup(
    bundleDict=[
        maf.metricBundles.MetricBundle(
            metric=maf.metrics.MedianMetric("seeingFwhmEff"),
            slicer=maf.slicers.TimeIntervalSlicer(interval_seconds=3600),
            constraint="",
            plotFuncs=[maf.plots.MonthHourglassPlot(10, 2023)],
        )
    ],
    dbObj=ops_db,
    outDir=maf_output_dir,
    resultsDb=results_db,
    dbTable="SummaryAllProps",
)
bundle_group.runAll()
bundle_group.plotAll(closefigs=False)

--Return--
None
> <ipython-input-12-81ba848d3d61>(1)<module>()
----> 1 breakpoint()
      2 logger.info("Starting hourglass")
      3 bundle_group = maf.metricBundles.MetricBundleGroup(
      4     bundleDict=[
      5         maf.metricBundles.MetricBundle(



ipdb>  b map.plotters.MonthHourglassPlot._plot


*** The specified object 'map.plotters.MonthHourglassPlot._plot' is not a function or was not found along sys.path.


ipdb>  b maf.plot.MonthHouglassPlot._plot


*** The specified object 'maf.plot.MonthHouglassPlot._plot' is not a function or was not found along sys.path.


...

<class 'matplotlib.figure.Figure'>


ipdb>  p intervals.__class__


<class 'pandas.core.frame.DataFrame'>


ipdb>  p intervals


                mjd  duration  value
0      59853.000000       1.0    NaN
1      59853.041667       1.0    NaN
2      59853.083333       1.0    NaN
3      59853.125000       1.0    NaN
4      59853.166667       1.0    NaN
...             ...       ...    ...
87691  63506.791666       1.0    NaN
87692  63506.833333       1.0    NaN
87693  63506.875000       1.0    NaN
87694  63506.916666       1.0    NaN
87695  63506.958333       1.0    NaN

[87696 rows x 3 columns]


In [ ]:
import numpy as np
np.array([]).__class__

In [ ]:
logger.info("Finished hourglass")

### Hour Angle by visit

Use the `VisitIntervaleSlicer` to divide the survey into time segments each of which covers one visit, and calculate the hour angle in each segment.
Make an hourglass plot of the result for October of 2023.

Set the color map to be diverging, and force the color limits such that 0 (transiting) is gray, while positive and negative hour angle are red and blue.

In [ ]:
logger.info("Starting hourglass")
bundle_group = maf.metricBundles.MetricBundleGroup(
    bundleDict=[
        maf.metricBundles.MetricBundle(
            metric=maf.metrics.MedianMetric("HA"),
            slicer=maf.slicers.VisitIntervalSlicer(),
            constraint="",
            plotDict={"cmap": plt.get_cmap("coolwarm"), "color_limits": (-4.5, 4.5)},
            plotFuncs=[maf.plots.MonthHourglassPlot(7, 2025)],
        )
    ],
    dbObj=ops_db,
    outDir=maf_output_dir,
    resultsDb=results_db,
    dbTable="SummaryAllProps",
)
bundle_group.runAll()
bundle_group.plotAll(closefigs=False)

In [ ]:
logger.info("Finished hourglass")

### Filter metric

In [ ]:
# This could have been the Pass metric instead .. (just returns the values in a given set of columns)
class FilterUseMetric(maf.metrics.BaseMetric):
    """Metric to classify visits by filter"""

    def __init__(self, filterCol="filter", **kwargs):
        self.filterCol = filterCol
        super().__init__(col=[filterCol], metricDtype="object", **kwargs)

    def run(self, dataSlice, slicePoint=None):  # pylint: disable=invalid-name
        """Run the metric.

        Parameters
        ----------
        dataSlice : numpy.NDarray
           Values passed to metric by the slicer, which the metric will use to calculate
           metric values at each slicePoint.
        slicePoint : Dict
           Dictionary of slicePoint metadata passed to each metric.
           E.g. the ra/dec of the healpix pixel or opsim fieldId.

        Returns
        -------
        str
            use at each slicePoint.
        """
        return dataSlice[self.filterCol]

In [ ]:
logger.info("Starting hourglass")
bundle_group = maf.metricBundles.MetricBundleGroup(
    bundleDict=[
        maf.metricBundles.MetricBundle(
            metric=FilterUseMetric(),
            slicer=maf.slicers.VisitIntervalSlicer(),
            constraint="night < 600",
            plotDict={
                "cmap": plt.get_cmap("Set1"),
                "assigned_colors": OrderedDict(
                    [("u", 1), ("g", 2), ("r", 4), ("i", 7), ("z", 0), ("y", 3)]
                ),
                "title": "Filter usage for April 2023",
                "legend_ncols": 1,
                "legend_loc": (1.01, 0.5),
                "legend_bbox_to_anchor": (1.01, 0.0),
                "legend": True,
            },
            plotFuncs=[maf.plots.MonthHourglassCategoricalPlot(4, 2023)],
        )
    ],
    dbObj=ops_db,
    outDir=maf_output_dir,
    resultsDb=results_db,
    dbTable="SummaryAllProps",
)
bundle_group.runAll()
bundle_group.plotAll(closefigs=False)

In [ ]:
logger.info("Finish hourglass")

### Time use by sets of adjacent visits with common "note" values

"Note" values in the database indicate how the scheduler selected visits. Divide the survey into time intervals of consecutive visits with common notes, and assign color codes according to the filters and program indicated by the note. 

#### One month

In [ ]:
logger.info("Starting hourglass")
bundle_group = maf.metricBundles.MetricBundleGroup(
    bundleDict=[
        maf.metricBundles.MetricBundle(
            metric=maf.metrics.UseMetric(),
            slicer=maf.slicers.BlockIntervalSlicer(),
            constraint="",
            plotDict={
                "title": "Time usage for October, 2023",
                "legend_elements": (
                    "wide with u, g, or r",
                    "wide with only IR",
                    "greedy",
                    "ECDFS",
                    "EDFS",
                    "ELAISS1",
                    "COSMOS",
                    "COSMOS-transit",
                    "XMM-LSS",
                    "moon",
                ),
            },
            plotFuncs=[maf.plots.MonthHourglassUsePlot(2, 2024)],
        )
    ],
    dbObj=ops_db,
    outDir=maf_output_dir,
    resultsDb=results_db,
    dbTable="SummaryAllProps",
)
bundle_group.runAll()
bundle_group.plotAll(closefigs=False)

In [ ]:
logger.info("Finished hourglass")

In this plot, the red represents scheduled "blobs" that include only exposures in IR (i, z, or y) filters, and the blue scheduled blobs with at least some exposures with filters covering shorter wavelengths. The orange shows times during which the "greedy" algorithm was used by the scheduler. Horizontal bars with other colors indicate times during which DDF fields were observed. The thick slanted yellow line shows the times of the transit of the moon, while the dotted yellow lines show moon rise and set. Slanted lines of other colors show  the transit times of each DDF field, with the same color coding as used for the bars that indicate when the DDFs are observed. The vertical axis indicates the day of the month on which the night starts, and the horizontal axis the time relative to local solar midnight (solar anti-transit). A white background indicates day time; black, full night; and different shades of gray, civil, nautical, and astronomical twilight.

If you wish to show the time relative to local civil midnight, you can pass the `solar_time=False` to the hourglass plotter:

In [ ]:
logger.info("Starting hourglass")
bundle_group = maf.metricBundles.MetricBundleGroup(
    bundleDict=[
        maf.metricBundles.MetricBundle(
            metric=maf.metrics.UseMetric(),
            slicer=maf.slicers.BlockIntervalSlicer(),
            constraint="",
            plotDict={
                "title": "Time usage for April, 2024",
                "xMin": -6.5,
                "xMax": 8.5,
                "legend_elements": (
                    "wide with u, g, or r",
                    "wide with only IR",
                    "greedy",
                    "ECDFS",
                    "EDFS",
                    "COSMOS" "ELAISS1",
                    "XMM-LSS",
                    "moon",
                ),
            },
            plotFuncs=[maf.plots.MonthHourglassUsePlot(4, 2024, solar_time=False)],
        )
    ],
    dbObj=ops_db,
    outDir=maf_output_dir,
    resultsDb=results_db,
    dbTable="SummaryAllProps",
)
bundle_group.runAll()
bundle_group.plotAll(closefigs=False)

Note the jump at the end of daylight savings time.

#### One Year

In [ ]:
%%time
logger.info("Starting hourglass")
bundle_group = maf.metricBundles.MetricBundleGroup(
    bundleDict=[
        maf.metricBundles.MetricBundle(
            metric=maf.metrics.UseMetric(),
            slicer=maf.slicers.BlockIntervalSlicer(),
            constraint="",
            plotDict={'figsize': (15, 10), 'title': "Time usage for 2023",
                      'legend_elements': ('wide with u, g, or r',
                                          'wide with only IR',
                                          'greedy',
                                          'ECDFS',
                                          'EDFS',
                                          'COSMOS'
                                          'ELAISS1',
                                          'XMM-LSS',
                                          'moon')},
            plotFuncs=[maf.plots.YearHourglassUsePlot(2023)],
        )
    ],
    dbObj=ops_db,
    outDir=maf_output_dir,
    resultsDb=results_db,
    dbTable="SummaryAllProps",
)
bundle_group.runAll()
bundle_group.plotAll(closefigs=False)

#### Whole survey

Uncomment to stop the notebook execution here so it completes in reasonable time:

In [ ]:
# assert False

Making plots for all years in the survey is slow, usually taking between 15 and 20 minutes on my test node.
This is done using separate instances of plotter objects, rather than just one, because the plot gets too compressed (or too big) otherwise.

In [ ]:
%%time
logger.info("Starting hourglass")
bundle_group = maf.metricBundles.MetricBundleGroup(
    bundleDict=[
        maf.metricBundles.MetricBundle(
            metric=maf.metrics.UseMetric(),
            slicer=maf.slicers.BlockIntervalSlicer(),
            constraint="",
            plotDict={'figsize': (15, 10),
                      'legend_elements': ('wide with u, g, or r',
                                          'wide with only IR',
                                          'greedy',
                                          'ECDFS',
                                          'EDFS',
                                          'COSMOS'
                                          'ELAISS1',
                                          'XMM-LSS',
                                          'moon')},
            plotFuncs=[maf.plots.YearHourglassUsePlot(y) for y in range(2022, 2033)],
        )
    ],
    dbObj=ops_db,
    outDir=maf_output_dir,
    resultsDb=results_db,
    dbTable="SummaryAllProps",
)
bundle_group.runAll()
bundle_group.plotAll(closefigs=False)

In [ ]:
logger.info("Finished hourglass")

In [ ]:
2